In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14422,2022-01-10T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1160,143,1303,117337,...,DECEDUTI: N. 1 IL 10/01/22 - N. 7 IL 09/01/22 ...,15.0,NaN,NaN,428094.0,29059.0,3995179.0,4854033.0,ITG,ITG1
14423,2022-01-10T17:00:00,ITA,9,Toscana,43.769231,11.255889,1084,113,1197,172700,...,NaN,5.0,NaN,NaN,411241.0,93485.0,5644736.0,4030631.0,ITI,ITI1
14424,2022-01-10T17:00:00,ITA,10,Umbria,43.106758,12.388247,205,11,216,33639,...,Si fa presente che 12 dei ricoveri non UTI app...,1.0,NaN,NaN,92591.0,21498.0,1388378.0,1687308.0,ITI,ITI2
14425,2022-01-10T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,46,6,52,5097,...,NaN,0.0,NaN,NaN,14520.0,6074.0,129136.0,253708.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14422,2022-01-10T17:00:00,19,Sicilia,1160,143,1303,117337,118640,6863,7803,...,457153,8849212,3311181.0,15.0,428094.0,29059.0,3995179.0,4854033.0,ITG,ITG1
14423,2022-01-10T17:00:00,9,Toscana,1084,113,1197,172700,173897,-1151,5790,...,504726,9675367,3830290.0,5.0,411241.0,93485.0,5644736.0,4030631.0,ITI,ITI1
14424,2022-01-10T17:00:00,10,Umbria,205,11,216,33639,33855,-118,1063,...,114089,3075686,575086.0,1.0,92591.0,21498.0,1388378.0,1687308.0,ITI,ITI2
14425,2022-01-10T17:00:00,2,Valle d'Aosta,46,6,52,5097,5149,275,450,...,20594,382844,117222.0,0.0,14520.0,6074.0,129136.0,253708.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2022-01-10', '2022-01-09')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2022-01-10T17:00:00,1,Piemonte,1845,140,1985,142436,144421,3023,8571,...,605597,11996694,3233363.0,7.0,414366.0,191231.0,4261019.0,7735675.0,ITC,ITC1
1,2022-01-10T17:00:00,2,Valle d'Aosta,46,6,52,5097,5149,275,450,...,20594,382844,117222.0,0.0,14520.0,6074.0,129136.0,253708.0,ITC,ITC2
2,2022-01-10T17:00:00,3,Lombardia,2999,246,3245,515967,519212,10084,17581,...,1554778,26523376,7094127.0,8.0,1089237.0,465541.0,13496188.0,13027188.0,ITC,ITC4
3,2022-01-10T17:00:00,5,Veneto,1437,201,1638,188561,190199,5740,7492,...,771246,20561250,2517564.0,13.0,621755.0,149491.0,8516970.0,12044280.0,ITH,ITH3
4,2022-01-10T17:00:00,6,Friuli Venezia Giulia,354,41,395,34484,34879,928,1601,...,183155,4712429,989190.0,4.0,147977.0,35178.0,2719597.0,1992832.0,ITH,ITH4
5,2022-01-10T17:00:00,7,Liguria,697,43,740,15823,16563,-249,2204,...,184696,3707817,1077232.0,0.0,167697.0,16999.0,2059439.0,1648378.0,ITC,ITC3
6,2022-01-10T17:00:00,8,Emilia-Romagna,2043,151,2194,209905,212099,12244,14194,...,691118,12151307,2453270.0,9.0,608761.0,82357.0,7543440.0,4607867.0,ITH,ITH5
7,2022-01-10T17:00:00,9,Toscana,1084,113,1197,172700,173897,-1151,5790,...,504726,9675367,3830290.0,5.0,411241.0,93485.0,5644736.0,4030631.0,ITI,ITI1
8,2022-01-10T17:00:00,10,Umbria,205,11,216,33639,33855,-118,1063,...,114089,3075686,575086.0,1.0,92591.0,21498.0,1388378.0,1687308.0,ITI,ITI2
9,2022-01-10T17:00:00,11,Marche,247,57,304,10584,10888,65,879,...,161283,2132157,1270552.0,3.0,161283.0,0.0,1704451.0,427706.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2022-01-09T17:00:00,1,Piemonte,1724,145,1869,139529,141398,5903,10240,...,597026,11933164,3223371.0,14.0,413318.0,183708.0,4253945.0,7679219.0,ITC,ITC1
1,2022-01-09T17:00:00,2,Valle d'Aosta,45,6,51,4823,4874,347,443,...,20144,382025,117026.0,0.0,14280.0,5864.0,129096.0,252929.0,ITC,ITC2
2,2022-01-09T17:00:00,3,Lombardia,2887,244,3131,505997,509128,33136,36858,...,1537197,26437009,7082512.0,27.0,1079769.0,457428.0,13458243.0,12978766.0,ITC,ITC4
3,2022-01-09T17:00:00,5,Veneto,1407,206,1613,182846,184459,11798,13973,...,763754,20511681,2513124.0,26.0,620467.0,143287.0,8499675.0,12012006.0,ITH,ITH3
4,2022-01-09T17:00:00,6,Friuli Venezia Giulia,345,37,382,33569,33951,2157,3100,...,181557,4701309,987995.0,4.0,147311.0,34246.0,2715799.0,1985510.0,ITH,ITH4
5,2022-01-09T17:00:00,7,Liguria,655,46,701,16111,16812,480,1532,...,166838,3698277,1075522.0,6.0,166838.0,0.0,2056176.0,1642101.0,ITC,ITC3
6,2022-01-09T17:00:00,8,Emilia-Romagna,1929,147,2076,197779,199855,15914,17698,...,676931,12112777,2446691.0,8.0,601082.0,75849.0,7522421.0,4590356.0,ITH,ITH5
7,2022-01-09T17:00:00,9,Toscana,1036,115,1151,173897,175048,11045,12454,...,498936,9645210,3820568.0,9.0,407941.0,90995.0,5629014.0,4016196.0,ITI,ITI1
8,2022-01-09T17:00:00,10,Umbria,197,13,210,33763,33973,-41,2581,...,113026,3062770,571787.0,3.0,92174.0,20852.0,1387710.0,1675060.0,ITI,ITI2
9,2022-01-09T17:00:00,11,Marche,255,54,309,10514,10823,-101,1884,...,160404,2126617,1266355.0,2.0,160404.0,0.0,1701745.0,424872.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2022-01-10T17:00:00,1,Piemonte,1845,140,1985,142436,144421,3023,8571,...,11996694,3233363.0,7.0,414366.0,191231.0,4261019.0,7735675.0,ITC,ITC1,4.341375e+06
1,2022-01-10T17:00:00,2,Valle d'Aosta,46,6,52,5097,5149,275,450,...,382844,117222.0,0.0,14520.0,6074.0,129136.0,253708.0,ITC,ITC2,1.255010e+05
2,2022-01-10T17:00:00,3,Lombardia,2999,246,3245,515967,519212,10084,17581,...,26523376,7094127.0,8.0,1089237.0,465541.0,13496188.0,13027188.0,ITC,ITC4,1.010397e+07
3,2022-01-10T17:00:00,5,Veneto,1437,201,1638,188561,190199,5740,7492,...,20561250,2517564.0,13.0,621755.0,149491.0,8516970.0,12044280.0,ITH,ITH3,4.907704e+06
4,2022-01-10T17:00:00,6,Friuli Venezia Giulia,354,41,395,34484,34879,928,1601,...,4712429,989190.0,4.0,147977.0,35178.0,2719597.0,1992832.0,ITH,ITH4,1.211357e+06
5,2022-01-10T17:00:00,7,Liguria,697,43,740,15823,16563,-249,2204,...,3707817,1077232.0,0.0,167697.0,16999.0,2059439.0,1648378.0,ITC,ITC3,1.543127e+06
6,2022-01-10T17:00:00,8,Emilia-Romagna,2043,151,2194,209905,212099,12244,14194,...,12151307,2453270.0,9.0,608761.0,82357.0,7543440.0,4607867.0,ITH,ITH5,4.467118e+06
7,2022-01-10T17:00:00,9,Toscana,1084,113,1197,172700,173897,-1151,5790,...,9675367,3830290.0,5.0,411241.0,93485.0,5644736.0,4030631.0,ITI,ITI1,3.722729e+06
8,2022-01-10T17:00:00,10,Umbria,205,11,216,33639,33855,-118,1063,...,3075686,575086.0,1.0,92591.0,21498.0,1388378.0,1687308.0,ITI,ITI2,8.802850e+05
9,2022-01-10T17:00:00,11,Marche,247,57,304,10584,10888,65,879,...,2132157,1270552.0,3.0,161283.0,0.0,1704451.0,427706.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2022-01-10T17:00:00,1,Piemonte,1845,140,1985,142436,144421,3023,8571,...,7735675.0,ITC,ITC1,4.341375e+06,Piemonte,1724,5824,145,628,99
1,2022-01-10T17:00:00,2,Valle d'Aosta,46,6,52,5097,5149,275,450,...,253708.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,45,99,6,33,0
2,2022-01-10T17:00:00,3,Lombardia,2999,246,3245,515967,519212,10084,17581,...,13027188.0,ITC,ITC4,1.010397e+07,Lombardia,2887,10457,244,1530,0
3,2022-01-10T17:00:00,5,Veneto,1437,201,1638,188561,190199,5740,7492,...,12044280.0,ITH,ITH3,4.907704e+06,Veneto,1407,6000,206,1000,0
4,2022-01-10T17:00:00,6,Friuli Venezia Giulia,354,41,395,34484,34879,928,1601,...,1992832.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,345,1277,37,175,0
5,2022-01-10T17:00:00,7,Liguria,697,43,740,15823,16563,-249,2204,...,1648378.0,ITC,ITC3,1.543127e+06,Liguria,655,1798,46,219,12
6,2022-01-10T17:00:00,8,Emilia-Romagna,2043,151,2194,209905,212099,12244,14194,...,4607867.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,1929,9001,147,889,0
7,2022-01-10T17:00:00,9,Toscana,1084,113,1197,172700,173897,-1151,5790,...,4030631.0,ITI,ITI1,3.722729e+06,Toscana,1036,5033,115,570,14
8,2022-01-10T17:00:00,10,Umbria,205,11,216,33639,33855,-118,1063,...,1687308.0,ITI,ITI2,8.802850e+05,Umbria,197,662,13,86,41
9,2022-01-10T17:00:00,11,Marche,247,57,304,10584,10888,65,879,...,427706.0,ITI,ITI3,1.518400e+06,Marche,255,1003,54,256,49


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2022-01-10T17:00:00,3,Lombardia,2999,246,3245,515967,519212,10084,17581,...,16.08,9970,-19277,86367,11615.0,61,7436,20.36,0.17400,15.39
1,2022-01-10T17:00:00,8,Emilia-Romagna,2043,151,2194,209905,212099,12244,14194,...,16.99,12126,-3504,38530,6579.0,16,1927,36.84,0.31774,15.47
2,2022-01-10T17:00:00,15,Campania,1085,77,1162,163766,164928,7169,13107,...,10.97,7114,1292,83720,15102.0,25,5913,15.66,0.22653,12.14
3,2022-01-10T17:00:00,12,Lazio,1518,197,1715,170046,171761,7270,9440,...,20.89,7195,-3388,60918,4299.0,19,2151,15.50,0.16094,10.57
4,2022-01-10T17:00:00,1,Piemonte,1845,140,1985,142436,144421,3023,8571,...,22.29,2907,-1669,63530,9992.0,13,5535,13.49,0.19743,13.95
5,2022-01-10T17:00:00,19,Sicilia,1160,143,1303,117337,118640,6863,7803,...,16.74,6821,-5146,31786,31465.0,23,917,24.55,0.15705,9.20
6,2022-01-10T17:00:00,5,Veneto,1437,201,1638,188561,190199,5740,7492,...,20.10,5715,-6481,49569,4440.0,24,1728,15.11,0.15266,15.72
7,2022-01-10T17:00:00,9,Toscana,1084,113,1197,172700,173897,-1151,5790,...,19.82,-1197,-6664,30157,9722.0,18,6923,19.20,0.15553,13.56
8,2022-01-10T17:00:00,16,Puglia,444,45,489,63792,64281,1951,2813,...,9.13,1930,-2091,39642,6877.0,2,860,7.10,0.07018,8.89
9,2022-01-10T17:00:00,13,Abruzzo,314,25,339,50078,50417,2057,2372,...,13.81,2036,-2258,47501,6140.0,1,313,4.99,0.18166,10.94


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,P.A. Trento,27,63,90,30.00
1,Friuli Venezia Giulia,41,134,175,23.43
2,Piemonte,140,488,628,22.29
3,Marche,57,199,256,22.27
4,Lazio,197,746,943,20.89
5,Veneto,201,799,1000,20.10
6,Toscana,113,457,570,19.82
7,Liguria,43,176,219,19.63
8,Valle d'Aosta,6,27,33,18.18
9,P.A. Bolzano,18,82,100,18.00


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))